<a href="https://colab.research.google.com/github/rizkynalfian/Clustering_mall_customer/blob/main/Mall_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Clustering**
Rizky Nur Alfian-DS0125

Analisis cluster atau clustering adalah sebuah proses mengkategorikan data ke dalam beberapa kelompok/ cluster sehingga anggota kelompok dalam satu cluster memiliki nilai kemiripan lebih tinggi daripada kelompok lain.

Di sini kami memodelkan pengelompokan Mall Customer Segmentation data. Kami menggunakan model ini untuk melakukan segmentasi pelanggan. Kami membedakan pelanggan menjadi jumlah grup yang optimal.

# **Import Libraries and Dataset**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering

In [ ]:
# Import Dataset
df = pd.read_csv ('https://raw.githubusercontent.com/rizkynalfian/Clustering_mall_customer/main/Mall_Customers.csv' , sep = ',')
df_customer = df
df.head()

In [ ]:
# Prints the Dataset Information
df_customer.info()

# **Data Preprocessing**

In [ ]:
# Prints Descriptive Statistics
df_customer.describe().transpose()

In [ ]:
# Prints Point Decriptive
df.describe(include=['O'])

# **Distribusi Data**

In [ ]:
fig = plt.figure(figsize = (16 , 6)) # create figure

ax0 = fig.add_subplot(1, 3, 1)
ax1 = fig.add_subplot(1, 3, 2) 
ax2 = fig.add_subplot(1, 3, 3)


sns.distplot(df_customer['Age'] , bins = 20, ax=ax0)
ax0.set_title("Distribusi Data 'Age'")

sns.distplot(df_customer['Annual Income (k$)'] , bins = 20, ax=ax1)
ax1.set_title ("Distribusi Data 'Annual Income (K$)'")

sns.distplot(df_customer['Spending Score (1-100)'] , bins = 20, ax=ax2)
ax2.set_title ("Distribusi Data 'Spending Score (1-100)'")


plt.show()

In [ ]:
# Pembagian data customer berdasarkan jenis kelamin
labels = ['Perempuan', 'Laki-laki']
size = df_customer['Gender'].value_counts()
colors = ['pink', 'blue']
explode = [0, 0.1]

# Pie chart
plt.rcParams['figure.figsize'] = (7, 7)
plt.pie(size, colors = colors, explode = explode, labels = labels, shadow = True, autopct = '%.2f%%')
plt.title('Jenis Kelamin', fontsize = 15)
plt.axis('off')
plt.legend()
plt.show()

**Melihat korelasi fitur**

In [ ]:
# Melihat korelasi fitur dengan heatmap
feature = df_customer.copy()
feature = df_customer.drop(['CustomerID'], axis=1)
df_customer.head()

feature['Gender'] = feature['Gender'].replace(['Male','Female'],[1,0])
correlation = feature.corr()
f , ax = plt.subplots(figsize = (10,10))
sns.heatmap(feature.corr(), annot=True).set_title('Korelasi Semua Fitur', size='16')

Grafik di atas menampilkan korelasi antara semua fitur dari kumpulan data customer, diagram heatmap ini mencerminkan fitur yang paling berkorelasi dengan warna oranye dan fitur yang paling sedikit berkorelasi dengan warna kuning.

Kami dapat melihat dengan jelas bahwa atribut ini tidak memiliki korelasi yang baik di antara mereka, oleh karena itu perlu dilanjutkan pencarian fitur yang memmiliki nilai korelasi yang paling tinggi.

In [ ]:
x = df_customer['Annual Income (k$)']
y = df_customer['Age']
z = df_customer['Spending Score (1-100)']

sns.lineplot(x, y, color = 'red')
sns.lineplot(x, z, color = 'blue')
plt.title('Income Pertahun vs Usia dan Spending Score', fontsize = 15)
plt.show()

Titik antara income tahunan dan usia di atas digambarkan dengan garis warna hijau, dan plot antara income tahunan dan spending score digambarkan dengan garis warna oranye.

Grafik diatas menunjukkan bagaimana korealasi fitur usia dan pengeluaran bervariasi dengan income pertahun.

## **Menentukan fitur untuk algoritma clustering.**

Tidak semua fitur akan dimasukan ke dalam algoritma clustering, kali ini kita hanya menggunakan fitur pendapatan per tahun dan spending score.

Pertama-tama kita harus definisikan dulu fitur ke dalam variabel baru.

In [ ]:
f = feature[['Annual Income (k$)', 'Spending Score (1-100)']]
f.head()

# **Scaling Data**

In [ ]:
# Melakukan penamaan untuk StandardScaler sebagai scaler
scaler = StandardScaler() 

# Fit Standardization
column_names = f.columns.tolist()
f[column_names] = scaler.fit_transform(f[column_names])
f.sort_index(inplace=True)
f

# **Visualisasi Model**
melakukan pemodelan terhadap data

In [ ]:
# Menentukan plot
sns.set() 
plt.rcParams['figure.figsize'] = (16, 9)

# Visualisasi data
sns.scatterplot(x='Annual Income (k$)', y='Spending Score (1-100)', data= f)
plt.title('Customer Segments')
plt.xlabel('Income Pertahun')
plt.ylabel('Spending Score')

# **K-Means Clustering Models**
K-Mean clustering merupakan salah satu “unsupervised machine learning algorithms” yang paling sederhana dan populer .Tujuan dari algoritma ini adalah untuk menemukan grup dalam data, dengan jumlah grup yang diwakili oleh variabel K. Variabel K sendiri adalah jumlah cluster yang kita inginkan.

In [ ]:
# Mengubah data frame menjadi numpy array
nump = f.to_numpy()
nump

# Elbow Method
from sklearn.cluster import KMeans
wcss = []
for i in range(1,11):
    kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=0)
    kmeans.fit(nump)
    wcss.append(kmeans.inertia_)
  
# Visualisasi 
plt.plot(range(1,11),wcss)
plt.title('Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('wcss')
plt.show()

In [ ]:
# Silhoutte method
for n_cluster in range(2, 11):
    kmeans = KMeans(n_clusters=n_cluster).fit(nump)
    label = kmeans.labels_
    sil_coeff = silhouette_score(nump, label, metric='euclidean')
    print('For n_clusters={}, The Silhouette Coefficient is {}'.format(n_cluster, sil_coeff))

Dari haril elbow method dan silhouette method diatas diketaui bahwa nilai K yang terbaik adalah 5, (k=5)

***Memodelkan K-Means Clustering***

In [ ]:
# Aplikasikan K-Means ke dalam data
kmeans = KMeans(n_clusters=5, init='k-means++', max_iter=300, n_init=10, random_state=0)
cluster = kmeans.fit_predict(nump)

# Visualisasi cluster untuk K=5
sns.scatterplot(x='Annual Income (k$)', y='Spending Score (1-100)', data= f)
plt.scatter(nump[cluster == 0, 0], nump[cluster == 0, 1], s = 50, label = 'Cluster 1', c= 'red')
plt.scatter(nump[cluster == 1, 0], nump[cluster == 1, 1], s = 50, label = 'Cluster 2', c= 'purple')
plt.scatter(nump[cluster == 2, 0], nump[cluster == 2, 1], s = 50, label = 'Cluster 3', c= 'blue')
plt.scatter(nump[cluster == 3, 0], nump[cluster == 3, 1], s = 50, label = 'Cluster 4', c= 'green')
plt.scatter(nump[cluster == 4, 0], nump[cluster == 4, 1], s = 50, label = 'Cluster 5', c= 'orange')
plt.scatter(kmeans.cluster_centers_[:,0], kmeans.cluster_centers_[:,1],s=200,marker='+', alpha=0.5, label='Centroids', c= 'black')
plt.title('Customer segments')
plt.xlabel('Annual Score')

plt.show()

**Analisis Model**

Dari hasil pemodelan K-Means diatas, kita dapat menyimpulkan bahwa secara keseluruhan, customer dapat dikelompokan menjadi 5 cluster yang masing-masing dibedakan dengan kelompok warna berbeda.

1. Kelompok warna Ungu, memiliki tingkat pendapatan rendah dan tingkat pengeluaran tinggi. Dalam keadaan ini customer dapat di golongkan menjadi customer Sangat Boros.

2. Kelompok warna Oranye, memiliki tingkat pendapatan rendah dan tingkat 
pengeluaran rendah. Dalam keadaan ini customer dapat di golongkan menjadi customer Hemat.

3. Kelompok warna Biru, memiliki tingkat pendapatan menengah dan tingkat pengeluaran menengah. Dalam keadaan ini customer dapat di golongkan menjadi customer Menengah.

4. Kelompok warna Hijau, memiliki tingkat pendapatan tinggi dan tingkat pengeluaran tinggi. Dalam keadaan ini customer dapat di golongkan menjadi customer Boros.

5. Kelompok warna merah, memiliki tingkat pendapatan tinggi dan tingkat pengeluaran rendah. Dalam keadaan ini customer dapat di golongkan menjadi customer Sangat Hemat.

**Mengaplikasikan Hasil ke Dalam Dataset**

Dari Dataset awal kita bisa tambahkan pengelompokan customer ke dalam data frame baru, lalu menyimpannya sebagai dataset hasil.

In [ ]:
# Menginput hasil clustering ke dataset
df_customer['cluster'] = cluster
df_customer['cluster'] = df_customer['cluster'].replace([0, 1, 2, 3, 4],['Sangat Hemat', 'Sangat Boros', 'Menengah', 'Boros', 'Hemat'])

# Menyimpan dataset baru sebagai hasil
df_customer.to_csv('_clusters.csv', index=False)

# Menampilkan dataframe baru
df_customer

**Hierarchical Clustering**
Pengelompokan hierarki (juga disebut analisis klaster hierarki atau HCA) adalah metode analisis klaster yang berupaya membangun hierarki klaster. Endpoint adalah sekumpulan cluster, di mana setiap cluster berbeda satu sama lain, dan objek dalam setiap cluster secara luas mirip satu sama lain.

In [ ]:
dendrogram = sch.dendrogram(sch.linkage(nump, method = 'ward'))
plt.title('Dendrogam', fontsize = 15)
plt.xlabel('Customers')
plt.ylabel('Ecuclidean Distance')
plt.show()

**Visualisasi Cluster**

In [ ]:
# Mengaplikasikan Hierarchial Clustering ke daaset
hc = AgglomerativeClustering(n_clusters=5, affinity='euclidean', linkage='ward')
hcluster = hc.fit_predict(nump)

# Visualisasi Cluster untuk K=5
sns.scatterplot(x='Annual Income (k$)', y='Spending Score (1-100)', data= f)
plt.scatter(nump[cluster == 0, 0], nump[cluster == 0, 1], s = 50, label = 'Cluster 1', c= 'red')
plt.scatter(nump[cluster == 1, 0], nump[cluster == 1, 1], s = 50, label = 'Cluster 2', c= 'purple')
plt.scatter(nump[cluster == 2, 0], nump[cluster == 2, 1], s = 50, label = 'Cluster 3', c= 'blue')
plt.scatter(nump[cluster == 3, 0], nump[cluster == 3, 1], s = 50, label = 'Cluster 4', c= 'green')
plt.scatter(nump[cluster == 4, 0], nump[cluster == 4, 1], s = 50, label = 'Cluster 5', c= 'orange')
plt.scatter(kmeans.cluster_centers_[:,0], kmeans.cluster_centers_[:,1],s=200,marker='+', alpha=0.5, label='Centroids', c= 'black')

plt.style.use('fivethirtyeight')
plt.title('Hierarchial Clustering', fontsize = 15)
plt.xlabel('Annual Income')
plt.ylabel('Spending Score')
plt.legend()
plt.grid()
plt.show()